In [1]:
## Importing required libraries
import csv
import urllib2    
from collections import defaultdict
import time
from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import pandas as pd
import string

In [2]:
import bs4 as bs
from bs4 import SoupStrainer

In [4]:
## importing the list of 13f-hr links for 971 randomly selected mutual funds
df = pd.read_csv('random 13f 1.csv')
urls = df.weblink
CIK = df.CIK
date = df.Date

In [8]:
## Lists to populate
company_name = []
CUSIP = []
class_name = []
value = []
shares = []
sh_prn = []
CIK_out =[]
date_out = []

In [9]:
## Looping through all the 13f-hr links
for j in range(0,len(urls)):
    class MyOpener(FancyURLopener, object):
            version = choice(user_agents)


    myopener = MyOpener()
    page=myopener.open(urls[j])
    html = page.read()

    soup = bs.BeautifulSoup(html,"lxml")
    tab_rows = soup.find_all("tr")
    
    ## Getting the data from all the data rows in each 13f information table
    for i in range(0,len(tab_rows)):
        if (len(tab_rows[i].find_all("td",class_="FormData")) > 0):
            company_name.append(tab_rows[i].find_all("td",class_="FormData")[0].getText())
            CUSIP.append(tab_rows[i].find_all("td",class_="FormData")[2].getText())
            class_name.append(tab_rows[i].find_all("td",class_="FormData")[1].getText())
            value.append(tab_rows[i].find_all("td",class_="FormDataR")[0].getText())
            shares.append(tab_rows[i].find_all("td",class_="FormDataR")[1].getText())
            sh_prn.append(tab_rows[i].find_all("td",class_="FormData")[3].getText())
            CIK_out.append(CIK[j])
            date_out.append(date[j])

In [10]:
##Data frame containing the mutual fund holdings data
df_out = pd.DataFrame({'Date': date_out,'CIK': CIK_out,'Company': company_name, 'CUSIP': CUSIP, 'class_name':class_name, 'Value': value, 'Shares': shares, 'sh_prn': sh_prn })

In [12]:
##Writing the data out to use later
df_out.to_csv("mutual_fund_holdings.csv",index=False)

In [28]:
## Standardising the CUSIP format
df_out.CUSIP = df_out.CUSIP.str.upper()

In [67]:
## Reading the CUSIP lookup table
cusip = pd.read_csv('SP 500 CUSIP.csv')
cusip = cusip[["Ticker symbol","Security","CUSIP"]]

In [68]:
## Merging the mutual fund holding with S&P 500 CUSIPs to get a subset for only S&P 500 companies
sp500_mutual = pd.merge(df_out, cusip, on='CUSIP', how='inner', indicator=True)

In [69]:
## Subsetting for rows with share values
sp500_mutual = sp500_mutual[sp500_mutual['sh_prn'].str.contains("SH")]

In [76]:
## Converting the Shares data to numeric
sp500_mutual['Shares'] = pd.to_numeric(sp500_mutual.Shares.str.replace(',',''))

## Getting the year column for roll ups
sp500_mutual['Date'] = pd.to_datetime(sp500_mutual['Date'])

sp500_mutual['Year'] = sp500_mutual['Date'].dt.year

In [80]:
## Aggregating the shares held by all mutual funds for each company and year
sp500_mutual_rollup1 = sp500_mutual.groupby(['CUSIP','Ticker symbol','Security','Year','CIK'],as_index=False)['Shares'].sum()

In [81]:
sp500_mutual_rollup1

,CUSIP,Ticker symbol,Security,Year,CIK,Shares
0,001055102,AFL,AFLAC Inc,2014,8565,102256
1,001055102,AFL,AFLAC Inc,2014,9634,37661
2,001055102,AFL,AFLAC Inc,2014,14661,2536
3,001055102,AFL,AFLAC Inc,2014,19617,1776635
4,001055102,AFL,AFLAC Inc,2014,36644,72584
5,001055102,AFL,AFLAC Inc,2014,40729,65000
6,001055102,AFL,AFLAC Inc,2014,44365,28916
7,001055102,AFL,AFLAC Inc,2014,51762,316770
8,001055102,AFL,AFLAC Inc,2014,89014,26511
9,001055102,AFL,AFLAC Inc,2014,108572,13657


In [88]:
sp500_mutual_shares = sp500_mutual_rollup1.groupby(['CUSIP','Ticker symbol','Security','Year'],as_index=False,group_keys = False,squeeze = True)['Shares'].sum()

In [92]:
sp500_mutual_shares.to_csv("sp500_mutual_shares.csv")